In [1]:
library(keras)
library(tidyverse)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
d = read_csv("http://cssbook.net/d/sentiment_headlines.csv")
head(d)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  id = col_double(),
  value = col_double(),
  lemmata = col_character()
)




id,value,lemmata
<dbl>,<dbl>,<chr>
10007,0,Rabobank voorspellen flink stijging hypotheekrente
10027,0,D66 willen reserve provincie aanspreken voor groei
10037,1,UWV dit jaar veel baan
10059,1,proost op geslaagd beursgang bols
10099,0,helft werknemer gaan na 65ste met pensioen
10101,1,Europa groeien voorzichtig dankzij laag energieprijs


In [3]:
text_vectorization <- layer_text_vectorization(max_tokens = 10000, output_sequence_length = 50)
text_vectorization %>% adapt(d$lemmata)

input <- layer_input(shape = c(1), dtype = "string")
output <- input %>% 
  text_vectorization() %>% 
  layer_embedding(input_dim = 10000 + 1, output_dim = 16) %>%
  layer_conv_1d(filters=128, kernel_size=3, activation='relu') %>%
  layer_global_max_pooling_1d() %>%
  layer_dense(units = 64, activation = "relu") %>%
  layer_dense(units = 1, activation = "tanh")

model <- keras_model(input, output)
model

Model
Model: "functional_1"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_1 (InputLayer)                [(None, 1)]                     0           
________________________________________________________________________________
text_vectorization (TextVectorizati (None, 50)                      0           
________________________________________________________________________________
embedding (Embedding)               (None, 50, 16)                  160016      
________________________________________________________________________________
conv1d (Conv1D)                     (None, 48, 128)                 6272        
________________________________________________________________________________
global_max_pooling1d (GlobalMaxPool (None, 128)                     0           
________________________________________________________________________________


In [6]:
d_train = d %>% slice_sample(n=4000)
d_test = d %>% anti_join(d_train)

model %>% compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = list('accuracy')
)

history <- model %>% fit(
  d_train$lemmata,
  d_train$value,
  epochs = 10,
  batch_size = 512,
  validation_split = 0.2,
  verbose=2
)

results <- model %>% evaluate(d_test$lemmata, d_test$value)
results

Joining, by = c("id", "value", "lemmata")



loss   accuracy 
-3.1938727  0.4422911